In [2]:
import folium
from folium.plugins import HeatMap
import pandas as pd

# Load your dataset
file_path = '0 Combined.xlsx'
vehicle_data = pd.read_excel(file_path)

# Function to create heatmap for a specific vehicle type
def create_heatmap_for_vehicle_type(vehicle_type, vehicle_data):
    # Filter data for the selected vehicle type
    filtered_data = vehicle_data[vehicle_data['Vehicle'] == vehicle_type]

    # Extract latitude and longitude data
    lat_long_data = filtered_data[['Trip Detai lLatitude', 'Trip Detail Longitude']]
    
    # Calculate the maximum number of visits to any location for the legend
    visit_counts = filtered_data.groupby(['Trip Detai lLatitude', 'Trip Detail Longitude']).size()
    max_visits = visit_counts.max()

    # Create a map centered around an average location of the filtered data
    if not lat_long_data.empty:
        map_center = [lat_long_data['Trip Detai lLatitude'].mean(), lat_long_data['Trip Detail Longitude'].mean()]
        m = folium.Map(location=map_center, zoom_start=13)

        # Add the heat map layer
        heat_data = [[row['Trip Detai lLatitude'], row['Trip Detail Longitude']] for index, row in lat_long_data.iterrows()]
        HeatMap(heat_data).add_to(m)

        # Create an HTML element for the dynamic legend with a more accurate gradient approximation
        legend_html = f'''
         <div style="position: fixed; 
         bottom: 50px; left: 50px; width: 130px; height: 140px; 
         border:2px solid grey; z-index:9999; font-size:14px;
         background-color: white; padding: 10px"
         >&nbsp; Visit Density <br>
         &nbsp; 1 visit &nbsp; <i style="background: #00FF00; width: 12px; height: 12px; display: inline-block;"></i> &nbsp; <br>
         &nbsp; {max_visits // 4} visits &nbsp; <i style="background: #7FFF00; width: 12px; height: 12px; display: inline-block;"></i><br>
         &nbsp; {max_visits // 2} visits &nbsp; <i style="background: #FFFF00; width: 12px; height: 12px; display: inline-block;"></i><br>
         &nbsp; {3 * max_visits // 4} visits &nbsp; <i style="background: #FF4500; width: 12px; height: 12px; display: inline-block;"></i><br>
         &nbsp; {max_visits} visits &nbsp; <i style="background: #FF0000; width: 12px; height: 12px; display: inline-block;"></i> &nbsp; 
         </div> 
         '''

        # Add the legend to the map
        m.get_root().html.add_child(folium.Element(legend_html))

        return m
    else:
        return "No data available for this vehicle type."

# Example usage
m = create_heatmap_for_vehicle_type('46', vehicle_data)

# To display in a Jupyter notebook
m
